DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four
#### Identifying the suspects' credit score
We received informations that the rogue agent has a good credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a good Credit_Mix


## Getting to know our data

* Age: a users age
* Occupation: a users employment field
* Annual_Income: a users annual income
* Monthly_Inh_Salary: the calculated salary received by a given user on a monthly basis
* Num_Bank_Accounts: the number of bank accounts possessed by a given user
* Num_Credit_Cards: the number of credit card given user possesses
* Interest_Rate: The interest rate on those cards (if multiple then its the average)
* Num_of_Loans: The number of loans of each user
* Delay_from_due_date: payment tardiness of user
* Num_of_Delayed_Payment: the count of delayed payments
* Changed_Credit_Limit: changes made to the credit limit for each user's account
* Num_Credit_Inquiries: number of credit inquiries
* Credit_Mix: The users credit score
* Outsting_Debt: Outstanding debt
* Credit_Utilization_Ratio: the percentage of borrowed money over borrowing allowance
* Payment_of_Min_Amount: does the user usually pay the minimal amount (categorical)
* Total_EMI_per_month: Monthly repayments to be made
* Amount_invested_monthly: The amout put in an investment fund by the user on a monthly basis
* Payment_Behaviour: the users payment behavior (categorical)
* Monthly_Balance: The users end of the month balance
* AutoLoan: If the user has an active loan for their vehicule
* Credit-BuilderLoan: If the user has a loan to increase their credit score
* DebtConsolidationLoan, HomeEquityLoan, MortgageLoan, NotSpecified, PaydayLoan, PersonalLoan, StudentLoan: different types of loans(categorical features)



In [2]:
# Import required packages
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/suspects.csv", index_col='Unnamed: 0').dropna()

In [ ]:
df.head()

,Age,Occupation,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Monthly_Balance,AutoLoan,Credit-BuilderLoan,DebtConsolidationLoan,HomeEquityLoan,MortgageLoan,NotSpecified,PaydayLoan,PersonalLoan,StudentLoan
0,23,Scientist,19114.12,1824.843333,3,4,3,4,3,7,...,186.266702,1,1,0,1,0,0,0,1,0
1,24,Scientist,19114.12,1824.843333,3,4,3,4,3,9,...,361.444004,1,1,0,1,0,0,0,1,0
3,24,Scientist,19114.12,4182.004291,3,4,3,4,4,5,...,343.826873,1,1,0,1,0,0,0,1,0
5,28,Teacher,34847.84,3037.986667,2,4,6,1,3,3,...,303.355083,0,1,0,0,0,0,0,0,0
8,35,Engineer,143162.64,4182.004291,1,5,8,3,8,1942,...,854.226027,2,0,0,0,0,1,0,0,0


In [29]:
suspects.head()

,Age,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,userID
0,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,317991
1,24.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,9.0,13.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,241892
3,24.0,19114.12,4182.004291,3.0,4.0,3.0,4.0,4.0,5.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,303376
5,28.0,34847.84,3037.986667,2.0,4.0,6.0,1.0,3.0,3.0,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,761992
8,35.0,143162.64,4182.004291,1.0,5.0,8.0,3.0,8.0,1942.0,7.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,373318


In [ ]:
df["Credit_Mix"].unique()

array(['Good', 'Standard', 'Bad'], dtype=object)

# 1. Preparing the data
## 1.1 Data cleaning
 Perform One-Hot Encoding over the "Occupation" feature.

 Then, perform Label Encoding over "Payment_of_Min_Amount" and "Payment_Behaviour".

 After performing the one-hot and label encoding, add the encoded features to the data frame and remove the corresponding categorical features.

In [7]:
# Your code here:
# Initialize

#Apply one-hot encoding to nominal variables
ohe = OneHotEncoder(sparse_output = False).set_output(transform = 'pandas')

# Apply label encoding to ordinal variables
le = LabelEncoder()

In [8]:
## Apply

# label
df['Payment_of_Min_Amount'] = le.fit_transform(df['Payment_of_Min_Amount'])
df['Payment_Behaviour'] = le.fit_transform(df['Payment_Behaviour'])

## one-hot
# transform
occupation_transformed = ohe.fit_transform(df[['Occupation']])

# concat with original df and drop original columns
df_new = pd.concat([df, occupation_transformed], axis = 1).drop(columns = 'Occupation')

In [ ]:
# on the suspects dataset


## 1.2 Dataset splitting and rescaling

a) Split the dataset in two, first X with your independent features and then y with the dependent feature **CreditMix**.

b) Split X and y into training and test sets. The training set should contain 80% of the observations, and the test set should contain the remaining 20%. Set random state equal to 42.

c) Then perform :
* Label Encoding over the **CreditMix** feature.
* A MinMaxScaller over all the independent features.

In [9]:
# Your code here
X = df_new.drop(columns = 'Credit_Mix')
y = pd.DataFrame(df_new['Credit_Mix'])

In [10]:
# Split
random_seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_seed)

In [11]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)


In [12]:
# Scale
y_train['Credit_Mix'] = le.fit_transform(y_train['Credit_Mix'])
y_test['Credit_Mix'] = le.transform(y_test['Credit_Mix'])

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
# on the whole dataset
X_scaled = scaler.fit_transform(X)

y_scaled = y.copy()
y_scaled['Credit_Mix'] = le.fit_transform(y['Credit_Mix'])

X_scaled_torch = torch.tensor(X_scaled, dtype=torch.float)
y_scaled_torch = torch.tensor(y_scaled.values.squeeze(), dtype=torch.long)


### 1.2.2 Final touches
Convert your datasets to `Torch tensors` of type `torch.float` for X and `torch.long` for y.

In [16]:
#Your code here:
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train.values.squeeze(), dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test.values.squeeze(), dtype=torch.long)


In [ ]:
print(X_train.size(), y_train.size())

torch.Size([23378, 42]) torch.Size([23378])


In [28]:
print(suspects.shape)

(714, 43)


In [61]:
print(le.classes_)

['Bad' 'Good' 'Standard']


# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
Set the following parameters:
* `hidden layer` : 1 with 150 nurons;
* `activation function` : ReLU
* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [17]:
# Define the neural network class
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)        # Linear transformation for hidden layer
        self.linear2 = nn.Linear(H1, D_out)       # Linear transformation for output layer
        self.activation = nn.ReLU()               # Activation function for hidden layer

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))   # Hidden layer: linear transformation + ReLU
        y_pred = self.linear2(y_pred)               # Output layer: linear transformation
        return y_pred


In [18]:
D_in = X_train.shape[1]
D_out = torch.unique(y_train).numel()
H1 = 150

In [19]:
# Model with 150 neurons
model1 = Net(D_in, H1, D_out)

In [20]:
# CEL loss
criterion = nn.CrossEntropyLoss()

## 2.2 Finding the best model:
Identify, amongst the following options the best parameters for your model:

* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
* `iterations` : 150, 250, 500, 1000
* `learning rate` : 0.00005, 0.001, 1.049, 12.031

Set `random seed` as torch.manual_seed(42).


_Hint: restart your runtime between each execution to ensure that previous neural networks dont interfere with your current one_

_You can evaluate your model based on it's accuracy over the test set_

In [ ]:
# SGD optimizer for finding the weights of the network
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-4)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Set a random seed for reproducibility
torch.manual_seed(42)

# Assuming the network 'Net' and datasets 'X_train_tensor', 'y_train_tensor', 'X_test_tensor', 'y_test_tensor' are already defined

# Define configurations for experiments
iterations_options = [150, 250, 500, 1000]
learning_rates = [0.00005, 0.001, 1.049, 12.031]

# Results storage
best_accuracy = 0
best_params = {}

# Define the dataset and data loader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Loop over the configurations
for epochs in iterations_options:
    for lr in learning_rates:
        # Define the model
        model = Net(D_in, H1, D_out)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=lr)

        # Training process
        for epoch in range(epochs):
            model.train()
            for data, target in train_loader:
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

        # Evaluation process
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = correct / total
        print(f'Epochs: {epochs}, Learning Rate: {lr}, Accuracy: {accuracy}')

        # Update the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'epochs': epochs, 'learning_rate': lr}

# Print the best model's parameters and accuracy
print(f'Best Accuracy: {best_accuracy}')
print(f'Best Parameters: {best_params}')


Epochs: 150, Learning Rate: 5e-05, Accuracy: 0.4545765611633875
Epochs: 150, Learning Rate: 0.001, Accuracy: 0.788366124893071
Epochs: 150, Learning Rate: 1.049, Accuracy: 0.857656116338751
Epochs: 150, Learning Rate: 12.031, Accuracy: 0.2309666381522669
Epochs: 250, Learning Rate: 5e-05, Accuracy: 0.4545765611633875
Epochs: 250, Learning Rate: 0.001, Accuracy: 0.808896492728828
Epochs: 250, Learning Rate: 1.049, Accuracy: 0.8388366124893071
Epochs: 250, Learning Rate: 12.031, Accuracy: 0.4545765611633875
Epochs: 500, Learning Rate: 5e-05, Accuracy: 0.5538066723695466
Epochs: 500, Learning Rate: 0.001, Accuracy: 0.8335329341317366
Epochs: 500, Learning Rate: 1.049, Accuracy: 0.8391787852865698
Epochs: 500, Learning Rate: 12.031, Accuracy: 0.4545765611633875
Epochs: 1000, Learning Rate: 5e-05, Accuracy: 0.7375534644995723
Epochs: 1000, Learning Rate: 0.001, Accuracy: 0.839863130881095
Epochs: 1000, Learning Rate: 1.049, Accuracy: 0.8696321642429427
Epochs: 1000, Learning Rate: 12.031, A

*Question 1:*

**Could we use BCELoss instead of CrossEntropyLoss?**

# 3. Predict over the suspects dataset

Now it's time to use the model to make predictions over the suspect dataset!

Use the following parameters:


* `hidden layer` : 1 with 150 neurons
* `output layer` : 3 neurons
* `optimizer` : [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
* `iterations` : 1000
* `learning rate` : 1.049

Set `random seed` as np.random.seed(42)



*Question 2:*

**Why does our model has 3 neurons in the output layer?**

In [52]:
# scaling on the whole dataset
X_scaled = scaler.fit_transform(X)

y_scaled = y.copy()
y_scaled['Credit_Mix'] = le.fit_transform(y['Credit_Mix'])

X_scaled_torch = torch.tensor(X_scaled, dtype=torch.float)
y_scaled_torch = torch.tensor(y_scaled.values.squeeze(), dtype=torch.long)

suspects_new = suspects.drop(columns = ['userID'])
suspects_new = suspects_new.drop(columns = ['Predicted_Class'])

suspects_new = pd.DataFrame(suspects_new)
suspects_new_scaled = scaler.transform(suspects_new)
suspects_new_np = suspects_new_scaled.astype('float32')  # Ensuring the data type is float32

# Convert numpy array to tensor
suspects_new_tensor = torch.tensor(suspects_new_np)



AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
suspects_new.head()

In [53]:
# Your code here
criterion = nn.CrossEntropyLoss()
random_seed = np.random.seed(42)
optimizer = torch.optim.SGD(model1.parameters(), lr=1.049)

losses1 = []
losses1_test = []

for t in range(1000):                # 1000 iterations

    # Forward pass: compute prediction on training set
    y_pred_fin = model1(X_scaled_torch)

    # Compute loss
    loss = criterion(y_pred_fin, y_scaled_torch)
    print(t, loss.item())
    losses1.append(loss.item())
    if torch.isnan(loss):
        break

    # Compute gradient
    optimizer.zero_grad()
    loss.backward()

    # Update
    optimizer.step()

    # Compute loss on test set
    losses1_test.append(criterion(model1(X_test), y_test).item())

0 1.1188501119613647
1 1.0518863201141357
2 1.0303517580032349
3 1.012447476387024
4 0.9934573769569397
5 0.9718789458274841
6 0.9472166895866394
7 0.9194234013557434
8 0.8888954520225525
9 0.8563928604125977
10 0.8230143785476685
11 0.7899256348609924
12 0.7581902742385864
13 0.728546142578125
14 0.7014028429985046
15 0.6769676804542542
16 0.655630350112915
17 0.640725314617157
18 0.6608588099479675
19 0.9400292634963989
20 2.1192266941070557
21 1.6618412733078003
22 1.112278938293457
23 0.9833551645278931
24 0.8517116904258728
25 0.8055447936058044
26 0.7747761607170105
27 0.7461743950843811
28 0.7257595658302307
29 0.7044364213943481
30 0.7045038938522339
31 0.7053259015083313
32 0.7892308831214905
33 0.7752671241760254
34 0.9487839937210083
35 0.6574645042419434
36 0.6592939496040344
37 0.6310534477233887
38 0.6697141528129578
39 0.653014600276947
40 0.7528970837593079
41 0.6604349613189697
42 0.7517648339271545
43 0.6197376847267151
44 0.6611361503601074
45 0.5980193614959717
46 0

In [54]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Assume X_train and y_train are already defined and preprocessed
# X_train = ... (your training feature matrix)
# y_train = ... (your training label array)

# Define the network dimensions
D_in = X_scaled_torch.shape[1]  # Number of input features
H1 = 150                 # Number of neurons in the hidden layer
D_out = 3                # Number of output classes

# Create an instance of the Net class
model = Net(D_in, H1, D_out)

# Define the optimizer and the loss function
optimizer = optim.SGD(model.parameters(), lr=1.049)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()               # Clear gradients for the next train
    output = model(X_scaled_torch)             # Forward pass: compute the output class probabilities
    loss = criterion(output, y_scaled_torch)   # Compute the loss: difference between the predicted and actual class
    loss.backward()                     # Backward pass: compute the weight
    optimizer.step()                    # Optimizer step: update the weights of neurons

    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}/{1000}, Loss: {loss.item()}')

Epoch 1/1000, Loss: 1.1188501119613647
Epoch 101/1000, Loss: 0.4700949192047119
Epoch 201/1000, Loss: 0.37704798579216003
Epoch 301/1000, Loss: 0.3590754568576813
Epoch 401/1000, Loss: 0.36542001366615295
Epoch 501/1000, Loss: 0.3414454460144043
Epoch 601/1000, Loss: 0.3376882076263428
Epoch 701/1000, Loss: 0.33350852131843567
Epoch 801/1000, Loss: 0.3297913670539856
Epoch 901/1000, Loss: 0.32872503995895386


In [55]:
# Prediction
model.eval()
with torch.no_grad():
    predictions = model(suspects_new_tensor)
    predicted_classes = torch.argmax(predictions, dim=1)

# Now `predicted_classes` holds the predicted classes for the new dataset

# Convert the predicted classes tensor to a numpy array
predicted_classes_np = predicted_classes.cpu().numpy()

# Create a DataFrame from the numpy array
predicted_df = pd.DataFrame(predicted_classes_np, columns=['Predicted_Class'])

# Assuming you have another DataFrame `suspects_df` which contains the details of the suspects
# Make sure that `suspects_df` has the same number of rows as `predicted_df`
userID = suspects['userID']

userID.index = predicted_df.index

suspects_score = pd.concat([userID, predicted_df], axis = 1)

# Now `suspects_df` contains a new column 'Predicted_Class' which has the predicted classes merged with other suspect details
print(suspects_score)

     userID  Predicted_Class
0    317991                1
1    241892                1
2    303376                1
3    761992                1
4    373318                1
..      ...              ...
709  458293                2
710  218415                2
711  173906                2
712  178685                0
713  200865                0

[714 rows x 2 columns]


In [58]:
print(suspects_score['Predicted_Class'].value_counts())

Predicted_Class
2    291
1    253
0    170
Name: count, dtype: int64


Display the table with two columns: 'userID' and the corresponding predicted Credit_Mix.

In [66]:
predicted_classes

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
        1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 1,
        1, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2,
        2, 1, 2, 2, 2, 0, 0, 0, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 2, 2,
        2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 2,
        2, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 1, 1, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0,
        2, 0, 0, 2, 2, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0,
        0, 0, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 2, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 2,
        2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2,
        2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 2,
        2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,

In [67]:
predictions

tensor([[-5.5805,  3.8890,  1.9711],
        [-6.0005,  4.0094,  2.2322],
        [-5.9562,  4.0965,  2.0339],
        ...,
        [-0.5312, -3.8668,  4.4326],
        [ 4.5020, -5.1333,  0.8979],
        [ 5.9284, -8.4128,  2.5953]])

In [57]:
#Your code here
predicted_df

,Predicted_Class
0,1
1,1
2,1
3,1
4,1
...,...
709,2
710,2
711,2
712,0


As mentioned in the beginning, we have reasons to believe that the suspect had a very good credit score. But we must make no errors, because a lot is at stake. We must be consident in our predictions.

Therefore, we need to analyze not just the predicted category but also how certain the model is about each prediction. Display the probabilities of observations in the 'suspects' dataset falling within the given classes.

_Hint: you can display the probabilities simply as a dataframe, but for better overview you can use visualization tools_

In [63]:
# Your code here


import pandas as pd
import torch

# Assuming model and suspects_new_tensor are already defined and loaded

# Evaluation mode and prediction
model.eval()
with torch.no_grad():
    probabilities = torch.softmax(model(suspects_new_tensor), dim=1)  # Get class probabilities
    predicted_classes = torch.argmax(probabilities, dim=1)  # Get predicted class index

# Convert the predicted classes tensor to a numpy array
predicted_classes_np = predicted_classes.cpu().numpy()

# Decode the predicted classes using the inverse of label encoding
# Assuming 'le' is your LabelEncoder instance
decoded_classes = le.inverse_transform(predicted_classes_np)

# Create a DataFrame with decoded classes
predicted_df = pd.DataFrame({
    'Predicted_Credit_Score': decoded_classes,
    'Probability': probabilities.max(dim=1).values.cpu().numpy()  # Get max probability for each prediction
})

# Assuming 'suspects' DataFrame contains 'userID'
# Make sure that 'suspects_df' has the same number of rows as 'predicted_df'
userID = suspects['userID']
userID.reset_index(drop=True, inplace=True)  # Reset index to align with predicted_df

# Concatenate userID with the predictions and probabilities
suspects_score = pd.concat([userID, predicted_df], axis=1)

# Print the final DataFrame
print(suspects_score)


     userID Predicted_Credit_Score  Probability
0    317991                   Good     0.871848
1    241892                   Good     0.855316
2    303376                   Good     0.887179
3    761992                   Good     0.912380
4    373318                   Good     0.890139
..      ...                    ...          ...
709  458293               Standard     0.913451
710  218415               Standard     0.994114
711  173906               Standard     0.992817
712  178685                    Bad     0.973447
713  200865                    Bad     0.965547

[714 rows x 3 columns]


In [64]:
user_ids_search = [200865, 761992, 858566, 862880, 526987]

# Assuming 'Good' and 'Excellent' are considered good credit scores
good_scores = ['Good']

# Filter the DataFrame for these user IDs and good credit scores
filtered_df = suspects_score[(suspects_score['userID'].isin(user_ids_search)) & (suspects_score['Predicted_Credit_Score'].isin(good_scores))]

# Print the filtered DataFrame
print(filtered_df)

     userID Predicted_Credit_Score  Probability
3    761992                   Good     0.912380
636  526987                   Good     0.927306


*Question 3:*

**Which of the following suspects have a good credit mix according to your model's predictions?**


*   200865
*   761992
*   858566
*   862880
*   526987
